In [321]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [322]:
test = pd.read_csv('Data/test.csv')
train = pd.read_csv('Data/train.csv')
gs = pd.read_csv('Data/gender_submission.csv')

In [323]:
del gs['PassengerId']

In [324]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [325]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [326]:
mean_Age = (train['Age'].mean() + test['Age'].mean())/2
print(mean_Age)

29.985854004252303


In [327]:
median_Age = (train['Age'].median() + test['Age'].median())/2
print(median_Age)

27.5


In [328]:
Surv_train = train['Survived']
train.drop(['Name','Ticket','Embarked','PassengerId'], axis='columns', inplace=True)
test.drop(['Name','Ticket','Embarked','PassengerId'], axis='columns', inplace=True)

In [329]:
test = test.fillna(0)
train = train.fillna(0)

In [330]:
train.loc[train["SibSp"] == 8,"SibSp"] = 5
test.loc[test["SibSp"] == 8,"SibSp"] = 5

In [331]:
train.loc[train["Cabin"] != 0,"Cabin"] = 1
test.loc[test["Cabin"] != 0,"Cabin"] = 1

In [332]:
train.loc[train["Sex"] == "male","Sex"] = 0
train.loc[train["Sex"] == "female","Sex"] = 1
test.loc[test["Sex"] == "male","Sex"] = 0
test.loc[test["Sex"] == "female","Sex"] = 1

In [333]:
train.loc[train["Age"] == 0,"Age"] = mean_Age
test.loc[test["Age"] == 0,"Age"] = mean_Age

In [334]:
decision = ExtraTreesClassifier(n_estimators=800)

In [335]:
X_train, X_test, y_train, y_test = train_test_split(train, Surv_train, test_size=0.33, random_state=42)

In [336]:
decision.fit(X_train, y_train)
pred = decision.predict(X_test)

In [337]:
print('Accuracy: \n', accuracy_score(y_test, pred, normalize = True))

Accuracy: 
 1.0


In [338]:
print('Accuracy: \n', accuracy_score(gs, pred, normalize = True))

ValueError: Found input variables with inconsistent numbers of samples: [418, 295]